In [ ]:
job_description = """ 
"""

In [19]:
!pip install fpdf


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40714 sha256=58cad6a29c14042f9b5e2abb8cb5118b6b0e35033c025f876dcdbfb7bc51132f
  Stored in directory: c:\users\reddy\appdata\local\pip\cache\wheels\f9\95\ba\f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:

from groq import Groq
import os
import datetime
import subprocess
import time
from dotenv import load_dotenv
import json

load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
client = Groq(api_key=api_key)

In [ ]:
job_description = """
 About the job

Mercor is training models that predict job performance better than a human can. Our technology is so powerful that it’s trusted by customers ranging from startups making their first hires to the largest AI labs hiring thousands of experts to train LLMs.


We’re at an 8-figure revenue run rate, growing 50% MoM, and have raised $32M from investors like Benchmark, General Catalyst, Peter Thiel, Jack Dorsey, Adam D'Angelo, and Larry Summers.


---------------------------------------------------------------


Regardless of whether you have human data experience, we think that successful Machine Learning Engineers at Mercor should have a solid technical background, demonstrated through:


    2+ years of experience in back-end development (e.g., Python / Django)
    Strong understanding of database design, SQL, and commonly used algorithms
    Some experience with cloud platforms (GCP or AWS) and infrastructure management
    Experience working with LLMs and/or other transformer-based models such as BERT
    Experience with search or recommendation systems, data classification, or LTR models
    Able to quickly understand complex problems & rapidly develop minimum viable solutions
    Attention to detail, particularly in terms of producing high-quality, stable code that can support rapid scaling
    A strong belief in regularly testing and using the things you build, and a strong sense of ownership for everything you’ve developed





"""

In [2]:
import json

with open('../resume.json') as f:
    resume_json = json.load(f)

In [40]:
from groq import Groq
import os
import json
from fpdf import FPDF
from dotenv import load_dotenv

# Load API key from environment variables
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
if not api_key:
    raise ValueError("GROQ_API_KEY is missing from environment variables.")

client = Groq(api_key=api_key)

class PDF(FPDF):
    """Custom PDF class to handle cover letter formatting."""
    def header(self):
        # Only add the "Cover Letter" heading on the first page
        if self.page_no() == 1:
            self.set_font('Arial', 'B', 14)
            self.cell(0, 12, 'Cover Letter', align='C', ln=1)
            self.ln(10)  # Add some space after the heading

def generate_cover_letter(job_description: dict, resume: dict) -> None:
    """
    Generate a personalized cover letter based on a job description and resume.
    """
    # Extract candidate name
    
    # Define the LLM prompt
    prompt = f"""
    You are an expert cover letter writer. Write a personalized cover letter for the following job description and resume:

    Job Description:
    {json.dumps(job_description, indent=4)}

    Resume:
    {json.dumps(resume, indent=4)}

    The cover letter should in the below format
    - Begin with 
    'Dear Hiring Manager'
    - First paragraph: Introduce the candidate and their background, highlighting relevant experience and skills.
    - Second paragraph: Detail experiences and projects, aligning them with the job requirements. Highlight how the projects and experience align with the job role.
    - Third paragraph: Explain why the candidate is ideal for the job and the company.
    - Be professional, concise, and tailored to the job requirements.

    Thanking you,
    candidate_name
    """

    try:
        # Query the LLM to generate the cover letter
        completion = client.chat.completions.create(
            model="llama3-70b-8192",  # Replace with your actual model name
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2048,
            top_p=0.9,
            stream=False,
            stop=None,
        )

        # Extract the generated cover letter
        if completion.choices and completion.choices[0].message:
            cover_letter = completion.choices[0].message.content

            start_index = cover_letter.lower().find("dear")
            if start_index == -1:
                print('Error')
            else:
                cover_letter = cover_letter[start_index:]        

            # Generate a PDF from the cover letter
            pdf = PDF()
            pdf.set_auto_page_break(auto=True, margin=15)
            pdf.add_page()
            pdf.set_left_margin(20)
            pdf.set_right_margin(20)
            pdf.set_font('Arial', size=10)

            # Add the cover letter content
            for line in cover_letter.splitlines():
                pdf.multi_cell(0, 5, line, border=0, align='J')

            # Save the PDF
            output_file = "cover_letter.pdf"
            pdf.output(output_file)

            print(f"Cover letter successfully saved to '{output_file}'.")

        else:
            print("No response received from the LLM. Please try again.")

    except Exception as e:
        print(f"Error in LLM processing: {e}")


In [41]:
generate_cover_letter(job_description, resume_json)

Cover letter successfully saved to 'cover_letter.pdf'.
